<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex09/Beatriz_Celante/Tentativa_2_Atividade_9_Beatriz_Celante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
nome = "Beatriz Celante"
print(f'Meu nome é {nome}')

Meu nome é Beatriz Celante


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [ ]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.0 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [ ]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [ ]:
# Check which GPU we are using
!nvidia-smi

Tue Jun  7 18:56:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [ ]:
from typing import List
from pandas.core.common import flatten

'''
=
Refiz o MyDataset, a implementação dele estava incorreta e esse era o principal fator que gerava o overfitting

'''

def tokenize(text: str, tokenizer):
    # Recomenda-se usar o tokenizer.batch_encode_plus pois é mais rápido.
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

        self.toekenized = []

        #Tokeniza os textos e insere o token de início
        for t in texts:
          to_token = tokenize(f'[CLS] {t}', tokenizer)
          
          #Adiciona o PAD
          pad_size = max(0, 1 + self.max_seq_length - len(to_token))
          to_token = to_token + [tokenizer.vocab['[PAD]']] * max(0, pad_size)

          #Separa os tokens por pedaço - percorre de 0 até o tamanho máximo, com um step do tamanho máximo da sequência:  
          for i in range (0, len(to_token) - 1, max_seq_length):
            part_input = to_token[i*max_seq_length : (i + 1)*max_seq_length]
            part_target = to_token[1 + i*max_seq_length : 1 + (i + 1)*max_seq_length]

            #Adiciona a sequência tokenizada no vetor
            if i + max_seq_length < len(to_token):
              self.toekenized.append(to_token[i: i + max_seq_length + 1])
            else:
              self.toekenized.append(to_token[-max_seq_length - 1:])


    def __len__(self):
        return len(self.toekenized)

    def __getitem__(self, idx):
        output = self.toekenized[idx]
        return torch.LongTensor(output[:-1]), torch.LongTensor(output[1:])

## Testando se a implementação do MyDataset está correta

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [ ]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-07 18:56:47--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.135.128, 74.125.142.128, 172.253.117.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.135.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   273MB/s    in 4.3s    

2022-06-07 18:56:51 (276 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [ ]:
# Load datasets
max_seq_length = 9

train_examples = 50000
valid_examples = 500
test_examples = 500
texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 51000 lines.


In [ ]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 6186062
valid examples: 68157
test examples: 68599


## Implementando a atenção e a atenção de várias cabeças

Querry, key e value possuem o formato (batch_size, seq_lenght, n_features). Fazer a multiplicação do query com a key resulta na importância de cada elemento da sequência para o modelo, e essa multiplicação possui dimensões de (batch_size, seq_lenght, seq_lenght). Isso é o que chamamos de atenção. Essa ateção é normalizada usando softmax porque a soma de todos os pesos têm que dar 1 (probabilidade). 

In [ ]:
class Multi_Attention(nn.Module):
    def __init__(self, max_len, dim, n_heads):
      super().__init__()

      self.dim = dim
      self.n_heads = n_heads
      self.max_len = max_len
      self.dim

      self.projection_q = nn.Linear(dim, dim, bias=False)
      self.projection_k = nn.Linear(dim, dim, bias=False)
      self.projection_v = nn.Linear(dim, dim, bias=False)

      self.mask = torch.tril(torch.ones(max_len, max_len)).unsqueeze(0).to(device)

    def attention(self, q, v, k, mask):
      scores = torch.matmul(q, k.transpose(-2, -1))
      scores = scores/math.sqrt(self.dim) #Fator de escala - estava errado na outra versão
      scores = scores.masked_fill(mask.unsqueeze(1) == 0, float('-inf'))
      
      #Aplica a softmax e multiplica por v para calcular a probabilidade
      soft = nn.functional.softmax(scores, dim = -1)
      prob = torch.matmul(soft, v)
      prob = prob.transpose(1,2)
      return prob

    def forward(self, inputs, mask):

      self.batch_size = inputs.shape[0]

      #Nessa segunda versão, implemento a conexão residual

      res = inputs
      
      q = self.projection_q(inputs).reshape(self.batch_size, self.max_len, self.n_heads, (self.dim // self.n_heads))
      k = self.projection_k(inputs).reshape(self.batch_size, self.max_len, self.n_heads, (self.dim // self.n_heads))
      v = self.projection_v(inputs).reshape(self.batch_size, self.max_len, self.n_heads, (self.dim // self.n_heads))

      q = q.transpose(1, 2)
      k = k.transpose(1, 2)
      v = v.transpose(1, 2)

      out = self.attention(q, v, k, mask)
      out = out.reshape(self.batch_size, self.max_len, self.dim)

      out = out + res

      return out

In [ ]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        super().__init__()

        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length
        self.dim = dim
        self.n_layers = n_layers
        self.pad_token_id = pad_token_id
        self.n_heads = 4

        #Embedding 
        self.embedding = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)

        #Positional embedding
        self.pos_embedding = nn.Embedding(vocab_size, dim, padding_idx=pad_token_id)

        #Dropout
        self.dropout = nn.Dropout(p=0.1)

        #Projeções lineares
        self.projection_W0 = nn.Linear(dim, dim, bias=False)

        #Camadas lineares (feed forward)
        self.feed_forward = nn.Sequential(
        nn.Linear(dim, 2*dim),
        nn.ReLU(),
        nn.Linear(2*dim, vocab_size),
        nn.Dropout(p=0.1))

        #Camadas de normalização
        self.layer_norm = nn.LayerNorm(self.dim, eps=1e-6)

        #Multi-head attention (Tinha esquecido de implementar de acordo com o número de camadas passado)
        self.multihead = nn.ModuleList([Multi_Attention(self.max_seq_length, self.dim, self.n_heads)
                                                                for _ in range(n_layers)])


    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """

        self.batch_size = inputs.shape[0]

        
        x_embed = self.embedding(inputs) + self.pos_embedding(inputs)
        x_embed = self.dropout(x_embed)

        #Definindo a máscara
        mask = torch.tril(torch.ones(self.batch_size, self.max_seq_length, self.max_seq_length)).to(device)
        mask = mask.masked_fill(inputs.unsqueeze(1) == self.pad_token_id, 0)
        mask = mask.masked_fill(inputs.unsqueeze(2) == self.pad_token_id, 0)

        for layer in self.multihead:
          out = layer(x_embed, mask)
          out = self.projection_W0(out)
          out = self.dropout(out)
          out =  self.layer_norm(out)

        out = self.feed_forward(out)

        return out

## Teste o modelo com um exemplo

In [ ]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [ ]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 7694178


## Assert da Perplexidade


In [ ]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_len, vocab_size)
        target: a LongTensor of shape (batch_size, seq_len)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              30715
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [ ]:
max_examples = 150_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=512, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=512)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30835.21, valid ppl: 30437.04
10000 steps; 5120000 examples so far; train ppl: 862.86, valid ppl: 335.30
20000 steps; 10240000 examples so far; train ppl: 520.14, valid ppl: 271.22
30000 steps; 15360000 examples so far; train ppl: 457.31, valid ppl: 246.82
40000 steps; 20480000 examples so far; train ppl: 426.74, valid ppl: 233.05
50000 steps; 25600000 examples so far; train ppl: 407.16, valid ppl: 223.77
60000 steps; 30720000 examples so far; train ppl: 393.80, valid ppl: 216.31
70000 steps; 35840000 examples so far; train ppl: 382.79, valid ppl: 210.76
80000 steps; 40960000 examples so far; train ppl: 374.35, valid ppl: 206.30
90000 steps; 46080000 examples so far; train ppl: 367.62, valid ppl: 202.61
100000 steps; 51200000 examples so far; train ppl: 361.86, valid ppl: 199.09
110000 steps; 56320000 examples so far; train ppl: 356.99, valid ppl: 196.84
120000 steps; 61440000 examples so far; train ppl: 352.51, valid ppl: 194.05
130000 steps; 665

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 182.81023934259906


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [ ]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte
Eu gosto de comer pizza pois me faz parte da
Eu gosto de comer pizza pois me faz parte da vida
Eu gosto de comer pizza pois me faz parte da vida,
Eu gosto de comer pizza pois me faz parte da vida, mas
Eu gosto de comer pizza pois me faz parte da vida, mas não
Eu gosto de comer pizza pois me faz parte da vida, mas não é
Eu gosto de comer pizza pois me faz parte da vida, mas não é o
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é o
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é o que
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é o que não
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é o que não é
Eu gosto de comer pizza pois me faz parte da vida, mas não é o que é o que não é o
Eu gosto de comer pizza pois me faz parte da vida

In [ ]:
prompt = 'Todas as cartas de amor são ridículas. Não seriam'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Todas as cartas de amor são ridículas. Não seriam os
Todas as cartas de amor são ridículas. Não seriam os seus
Todas as cartas de amor são ridículas. Não seriam os seus filhos
Todas as cartas de amor são ridículas. Não seriam os seus filhos,
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o que
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o que não
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o que não é
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o que não é o
Todas as cartas de amor são ridículas. Não seriam os seus filhos, mas não é o que não é o que
Todas as cartas de a

In [ ]:
prompt = 'Muito prazer meu nome é otário vindo de outros'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Muito prazer meu nome é otário vindo de outros.
Muito prazer meu nome é otário vindo de outros. O
Muito prazer meu nome é otário vindo de outros. O que
Muito prazer meu nome é otário vindo de outros. O que o
Muito prazer meu nome é otário vindo de outros. O que o que
Muito prazer meu nome é otário vindo de outros. O que o que se
Muito prazer meu nome é otário vindo de outros. O que o que se refere
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu trabalho
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu trabalho é
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu trabalho é o
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu trabalho é o que
Muito prazer meu nome é otário vindo de outros. O que o que se refere ao seu trabalho é o q

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.